<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

# loading data from a csv fle
url_1 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/16Part1.csv'
url_2 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/16Part2.csv'

data_1 = pd.read_csv(url_1)
data_2 = pd.read_csv(url_2)

In [0]:
# [0] fillna, isin

data1 = data_1.dropna(subset=['Year'])
data1 = data1[data1['Year'] >= 1980]

# replace na to 0 in each column
cols = list(data1.columns)[6:]
for i in cols:
    data1[i] = data1[i].fillna(value=0)

# drop na
data2 = data_2.dropna()

# convert height and weight
def convHeight(x):
    """ convert height from Feet-Inch formt to cm """
    h_us = x.split('-')
    return round(int(h_us[0]) * 30.5 + int(h_us[1]) * 2.54)

data2 = data2.assign(height=data2.apply(lambda x: convHeight(x['height']), axis=1))
data2 = data2.assign(weight=data2.apply(lambda x: round(x['weight'] / 2.2046), axis=1))

# find dupplicated players and exclude players in dup_player
# DON'T TRY: find unique players and include players in unique players
#            unique players in certain years are duplicated players in other years
dup_player = data1.groupby(['Year', 'Player']).agg({'Age': 'nunique'})
dup_player.reset_index(inplace=True)
dup_player = dup_player[dup_player['Age'] > 1]
dup_player = dup_player.groupby('Player').agg({'Year': 'nunique'})
dup_player.reset_index(inplace=True)
data1 = data1[data1['Player'].isin(dup_player['Player']) == False]

# find number of years and exclue players with >20 or <3 condition
# NOT year counting, BUT year range (there might be empty years)
yr_min = data1.groupby('Player').agg({'Year': 'min'})
yr_max = data1.groupby('Player').agg({'Year': 'max'})
yr_outlier = pd.merge(yr_min, yr_max, left_index=True, right_index=True)
yr_outlier = yr_outlier.assign(period=yr_outlier.apply(lambda x:
                                                    x['Year_y'] - x['Year_x'],
                                                    # why not adding 1 (???)
                                                    axis=1))
yr_outlier.reset_index(inplace=True)
yr_outlier = yr_outlier[(yr_outlier['period'] > 20) | (yr_outlier['period'] < 3)]
data1 = data1[data1['Player'].isin(yr_outlier['Player']) == False]

# adding height and weight
data = pd.merge(data1, data2, how='inner', left_on='Player', right_on='name')

In [3]:
# [5]

data5 = data.groupby(['Tm', 'Year']).agg({'Age': 'mean'})
data_by_age = data5.sort_values(by='Age', ascending=False)
data_by_age.reset_index(inplace=True)
team = data_by_age.iloc[0]['Tm']
year = data_by_age.iloc[0]['Year']

player = data[(data['Year'] == year) & (data['Tm'] == team)].sort_values(by='Age',
                                              ascending=False).iloc[0]['Player']

print(f'Answer [5] : Oldest player = {player}')

Answer [5] : Oldest player = Tim Duncan


In [4]:
# [6] ttest_ind

from scipy import stats
import math

# define a player is a multi-player or not
data6 = data.groupby('Player').agg({'Pos': 'nunique', 'height': 'mean'})

single = data6[data6['Pos'] == 1]['height']
multi = data6[data6['Pos'] >= 2]['height']

t_val, p_val = stats.ttest_ind(single, multi, equal_var=False)

print(f'Answer [6] : P-value = {math.floor(p_val * 10000) / 10000:.4f}')

Answer [6] : P-value = 0.1676


In [0]:
# [7 ~ 8 data processing]

# count team number per year and player
team_cnt = data.groupby(['Year', 'Player']).agg({'Tm': 'nunique'})
team_cnt.reset_index(inplace=True)

# tag moved or not (if cnt > 1 then moved)
team_cnt = team_cnt.assign(mov=team_cnt.apply(lambda x: 1 if x['Tm'] > 1 else 0,
                                             axis=1))
#team_cnt[team_cnt['mov'] == 0].count()

# make mov_season and prev_season
data7 = pd.merge(data, team_cnt, how='inner', on=['Year', 'Player'])
mov_season = data7[data7['mov'] == 1]
prev_season = data7[data7['mov'] == 0]

In [6]:
# [7]

import math

three = mov_season.groupby(['Year', 'Player']).agg({'ThreeP': 'sum',
                                                    'ThreePA': 'sum'})
three = three.assign(success=three['ThreeP'] / three['ThreePA'])

ave_success = three[three['ThreePA'] >= 10]['success'].mean()

print(f'Answer [7] : Average success rate = {math.floor(ave_success * 10000) / 10000:.4f}')

Answer [7] : Average success rate = 0.3002


In [7]:
# [8] dummy variables, LinearRegression, mean_squared_error, fillna

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math

# transform Pos to dummy variables
dummy_Pos = pd.get_dummies(prev_season['Pos'], drop_first=True)
prev_season = pd.concat([prev_season, dummy_Pos], axis=1)

# create calculated columns
prev_season = prev_season.assign(suc2=prev_season['TwoP'] / prev_season['TwoPA'])
prev_season = prev_season.assign(suc3=prev_season['ThreeP'] / prev_season['ThreePA'])
prev_season = prev_season.assign(sucFT=prev_season['FT'] / prev_season['FTA'])

# fillna
for col in ['suc2', 'suc3', 'sucFT']:
    prev_season[col] = prev_season[col].fillna(value=0)

# create train set and test set
train = prev_season[prev_season['Year'] <= 2000]
test = prev_season[prev_season['Year'] >= 2001]

# creat train_X and test_X
train_X = train[['PF', 'PG', 'SF', 'SG', 'Age', 'MP', 'suc2', 'suc3', 'sucFT',
                 'AST', 'height', 'weight']]
test_X = test[['PF', 'PG', 'SF', 'SG', 'Age', 'MP', 'suc2', 'suc3', 'sucFT',
               'AST', 'height', 'weight']]

# create train_y and test_y
train_y = train['PTS']
test_y = test['PTS']

# train
lr = LinearRegression()
model = lr.fit(train_X, train_y)

# teset
pred = model.predict(test_X)

# find MSE
mse = mean_squared_error(test_y, pred)

print(f'Answer [8] : MSE = {math.floor(mse)}')

Answer [8] : MSE = 38384
